In [78]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D,AveragePooling2D
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import DenseNet121

In [79]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [80]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [81]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [82]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [83]:
weights_Add="/kaggle/input/vggweight/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG19(weights=weights_Add, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [84]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(5, activation='softmax')  
])

In [85]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_27 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,630,405 (82.51 MB)

 Trainable params: 1,606,021 (6.13 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [86]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 50s 536ms/step - accuracy: 0.6405 - loss: 1.7361 - val_accuracy: 0.7925 - val_loss: 0.5682
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 46s 524ms/step - accuracy: 0.7908 - loss: 0.5862 - val_accuracy: 0.8237 - val_loss: 0.4850
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 528ms/step - accuracy: 0.8275 - loss: 0.5026 - val_accuracy: 0.8527 - val_loss: 0.4391
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 528ms/step - accuracy: 0.8293 - loss: 0.4937 - val_accuracy: 0.8413 - val_loss: 0.4382
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 531ms/step - accuracy: 0.8410 - loss: 0.4568 - val_accuracy: 0.8340 - val_loss: 0.4789
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 531ms/step - accuracy: 0.8481 - loss: 0.4222 - val_accuracy: 0.8672 - val_loss: 0.3995
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 531ms/step - accuracy: 0.8631 - loss: 0.4163 - val_accuracy: 0.8320 - val_loss: 0.4701
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 530ms/step - accuracy: 0.8594 - loss: 0.4076 - val_accu